In [39]:
# Auto Load module
%load_ext autoreload
%autoreload 2
from CohortPCA import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of CohortPCA failed: Traceback (most recent call last):
  File "c:\Users\Wang Jiayao\Anaconda2\lib\site-packages\IPython\extensions\autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named model_selection
]


In [40]:
instance = CohortPCA()
instance.loadControlPanel()
instance.loadEigenValue()
#instance.ReduceSuperPop()
#instance.Plot("RGN")
#instance.MarkerWithKNN()
instance.MarkerWithSVM()

Fitting Model...
Done with Fitting, used 0.0660 secs.


c:\Users\Wang Jiayao\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
c:\Users\Wang Jiayao\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
c:\Users\Wang Jiayao\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  

In [41]:
from sklearn.model_selection import cross_validate

ImportError: No module named model_selection

In [ ]:
test = Validation("CaseAnnotationKNN.txt")
test.run("Test.Annotate.KNN")
test = Validation("CaseAnnotationSVM.txt")
test.run("Test.Annotate.SVM")    